In [1]:
import findspark
findspark.init("/usr/lib/spark")
from pyspark.sql import SparkSession
import pyspark.sql.functions as func 
import pulp
import __builtin__ as bt
import pandas as pd
from pyspark.sql.types import DecimalType, StructType, StructField, IntegerType, StringType
import decimal


In [2]:
from datetime import datetime
import subprocess
import uuid
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.types import DecimalType, StructType, StructField, IntegerType, StringType
import decimal
import re
import __builtin__ as bt

In [3]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
latest_date ='20181112'

offer_status = spark.read.csv("s3://stx-apollo-pr-datascience-internal/michael/test-cell/dash-offer-status-fix", sep='|', header=True)


measure1 = spark.read.csv("s3://stx-apollo-pr-datascience-internal/prod/optimization_v1.2/%s/measure/" % latest_date, sep="|") \
    .toDF( \
    "sid"
    , "cell_name"
    , "revenue"
    , "discount"
    , "stars_awarded"
    , "tier"
    , 'email_open_flag'
    , 'customer_opted_in_flag'
    , 'winner_flag'
    , 'number_redemptions')



measure = measure1.alias("a").join(offer_status
                               , [func.col("a.sid") == offer_status.sid,
                                  func.col("a.cell_name") == offer_status.variant_name]
                               , how='left') \
                               .select(func.col("a.*"), 'offer_status') \
                               .where("offer_status is null or offer_status = 'offer'")\
                               .drop('offer_status')

measure = measure.withColumn("revenue", measure["revenue"].cast(DecimalType(8, 2))) \
    .withColumn("discount", measure["discount"].cast(DecimalType(8, 2))) \
    .withColumn("stars_awarded", measure["stars_awarded"].cast(DecimalType(8, 2)))

measure.registerTempTable("measure")

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Load segment_lookup

segment_lookup = spark.read.csv("s3://stx-apollo-pr-datascience-internal/prod/optimization_v1.2/%s/segment_lookup/" % latest_date, sep="|") \
    .toDF(
    "segment_id",
    "offer_type",
    "segment",
    "frequency_label",
    "revenue_label",
    "optin_label",
    "highvalue_label",
    "transaction_label",
    "ampm_label")

segment_lookup_empty_str = segment_lookup.fillna("")
segment_lookup_empty_str.registerTempTable("segment_lookup_empty_str")

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Load offer_lookup

offer_lookup = spark.read.csv("s3://stx-apollo-pr-datascience-internal/prod/optimization_v1.2/%s/offer_lookup/" % latest_date, sep="|") \
    .toDF(
    'offer_id'
    , 'offer_type'
    , 'product_variant'
    , 'optin_required_flag'
    , 'reward1_reward'
    , 'reward2_reward'
    , 'reward3_reward'
    , 'hurdle1_int'
    , 'hurdle2_int'
    , 'hurdle3_int'
    , 'hurdle1_up'
    , 'hurdle2_up'
    , 'hurdle3_up'
    , 'duration'
    , 'qualifying_daypart'
    , 'product_category'
    , 'reminder_bool')
offer_lookup.registerTempTable("offer_lookup")

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Load test variants

variant_1temp = spark.read.csv("s3://stx-apollo-pr-datascience-internal/prod/optimization_v1.2/%s/variant/" % latest_date, sep="|") \
    .toDF(
    'name'
    , 'reward1_reward'
    , 'reward2_reward'
    , 'reward3_reward'
    , 'hurdle1_int'
    , 'hurdle2_int'
    , 'hurdle3_int'
    , 'hurdle1_up'
    , 'hurdle2_up'
    , 'product_variant'
    , 'optin_required_flag'
    , 'test_variant_flag'
    , 'relevant_local_control_cell'
    , 'start_date'
    , 'offer_type'
    , 'weekstartdate'
    , 'hurdle3_up'
    , 'duration'
    , 'qualifying_daypart'
    , 'product_category'
    , 'reminder_bool')

cols = variant_1temp.columns

## Additional logic to replicate old dash 7-day, streak 7-day and Product Dash variants as
## Dash 7-day 2-hurdle, Streak 7-day 3-hurdle, Product Dash 1-hurdle, respectively

temp_pdash = variant_1temp.filter(variant_1temp.offer_type == 'Product Dash').withColumn(
    "offer_type", func.lit("Product Dash 1-hurdle")).select(*cols)
temp_dash = variant_1temp.filter(variant_1temp.offer_type == 'Dash 7-day').withColumn("offer_type",func.lit("Dash 7-day 2-hurdle"))\
                .select(*cols)
temp_streak = variant_1temp.filter(variant_1temp.offer_type == 'Streak 7-day').withColumn(
    "offer_type", func.lit("Streak 7-day 3-hurdle")).select(*cols)

variant_temp_ = variant_1temp.unionAll(temp_dash).unionAll(temp_pdash).unionAll(temp_streak)

#### NEW CODE BEGINS HERE ####
# Get the distinct offer_types in the variants
offer_types = [str(offer[0]) for offer in variant_temp_.select('offer_type').distinct().collect()]
ctrl_frames = []

def reLabelControlVariants(df, offer_type):
    '''
    Desc: This helper function selects all of the test variants in an offer type and
          their relevant local control variants. The offer_type of the relevant local
          control cell is then remapped to the offer_type in question.
          This function will duplicate any existing control variants so a deduplication step is necessary.
    Input: - The dataframe of the variants UNLOADED to "s3://stx-apollo-pr-datascience-internal/prod/optimization_v1.2/{datetime}/variant"
           - offer_type as a String, e.g. "Dash 7-day 2-hurdle", "Streak 7-day 3-hurdle"
    Output: A dataframe of the control variants needed for all of the test variants in the offer_type
    '''
    test_variants = df.where(" offer_type = '{0}' and test_variant_flag = 1 ".format(offer_type) )
    local_ctrl_names = [str(lc[0]) for lc in test_variants.select('relevant_local_control_cell').distinct().collect()]
    ctrl_variants = df.filter( func.col('name').isin(local_ctrl_names) ) \
                                        .drop('offer_type') \
                                        .selectExpr("*",
                                                     " '{0}' as offer_type".format(offer_type)) \
                                        .select(*test_variants.columns)
    return ctrl_variants

#Apply the offer relabeling function to all of the control variants
ctrl_frames = [reLabelControlVariants(variant_temp_, offer_type) for offer_type in offer_types]

def unionMany(*dfs):
    '''
    Description: This helper function unions many dataframes together.
    '''
    return reduce(DataFrame.unionAll, dfs)

# Unional all of the new control variants with the original variants
variant_temp_reduced = unionMany(*ctrl_frames).unionAll(variant_temp_)


# ctrl_frames =  reLabelControlVariants(variant_temp_, offer_types[0])
#
# for offer_type in offer_types[1:]:
#     ctrl_frames = ctrl_frames.union(reLabelControlVariants(variant_temp_, offer_type))
#
# variant_temp_reduced = ctrl_frames.union(variant_temp_)

# Remove any duplicated variants
variant_temp = variant_temp_reduced.dropDuplicates()
#### NEW CODE STOPS HERE ####

variant_temp.registerTempTable("variant_temp")

variant = spark.sql('''
	select
	a.*
	,b.offer_id
	from variant_temp a
	left join offer_lookup b
	on a.offer_type <=> b.offer_type
		and a.product_variant <=> b.product_variant
		and a.optin_required_flag <=> b.optin_required_flag
		and a.reward1_reward <=> b.reward1_reward
		and a.reward2_reward <=> b.reward2_reward
		and a.reward3_reward <=> b.reward3_reward
		and a.hurdle1_int <=> b.hurdle1_int
		and a.hurdle2_int <=> b.hurdle2_int
		and a.hurdle3_int <=> b.hurdle3_int
		and a.hurdle1_up <=> b.hurdle1_up
		and a.hurdle2_up <=> b.hurdle2_up
		and a.hurdle3_up <=> b.hurdle3_up
		and a.duration <=> b.duration
		and a.qualifying_daypart <=> b.qualifying_daypart
		and trim(a.product_category) <=> trim(b.product_category)
		and a.reminder_bool <=> b.reminder_bool
''')

user_seg_new = spark.read.csv("s3://stx-apollo-pr-datascience-internal/prod/optimization_v1.2/%s/user_seg_new/" % latest_date,header=True,sep="|")
model_user = user_seg_new.groupBy("segment_id") \
    .count() \
    .withColumnRenamed('count', 'user_count')

user_seg = spark.read.csv("s3://stx-apollo-pr-datascience-internal/prod/optimization_v1.2/%s/user_seg/" % latest_date,header=True,sep="|")

In [ ]:
user_seg.filter(func.col("segment_id") == 1283).groupby("segment_id","offer_id").count().sort("offer_id").show(100,False)

In [ ]:
user_full_ = user_seg.join(variant, [user_seg.variant_name == variant.name])\
    .join(measure, [user_seg.sid == measure.sid, user_seg.variant_name == measure.cell_name])\
    .select(user_seg.sid,
            user_seg.offer_id,
            user_seg.segment_id,
            variant.start_date,
            variant.test_variant_flag,
            measure.discount,
            measure.revenue).distinct()
user_revenue = user_full_.groupby("segment_id","offer_id","test_variant_flag")\
            .agg(func.avg("revenue").alias("avg_revenue"),
                 func.stddev_samp("revenue").alias("std_revenue"),
                 func.count("revenue").alias("cust_count")).sort("offer_id")
user_revenue.filter(func.col("segment_id") == 1283).show(100,False)

In [4]:
user_full_ = user_seg.join(variant, [user_seg.variant_name == variant.name])\
    .join(measure, [user_seg.sid == measure.sid, user_seg.variant_name == measure.cell_name])\
    .join(segment_lookup, [user_seg.segment_id == segment_lookup.segment_id])\
    .select(user_seg.sid,
            user_seg.offer_id,
            user_seg.segment_id,
            variant.start_date,
            variant.test_variant_flag,
            segment_lookup.offer_type,
            segment_lookup.frequency_label,
            measure.discount,
            measure.revenue).distinct()
user_full_ = user_full_.fillna("na",['frequency_label','offer_id'])

In [6]:
path ="s3://stx-apollo-pr-datascience-internal/eosoba/optimization_v1.2/{0}/user_full".format(latest_date)
user_full_.write.parquet(path,mode="overwrite")

In [7]:
path ="s3://stx-apollo-pr-datascience-internal/eosoba/optimization_v1.2/{0}/user_full".format(latest_date)
user_full_ = spark.read.parquet(path)

In [ ]:
user_revenue = user_full_.groupby("segment_id","offer_id","test_variant_flag")\
            .agg(func.avg("revenue").alias("avg_revenue"),
                 func.stddev_samp("revenue").alias("std_revenue"),
                 func.count("revenue").alias("cust_count")).sort("offer_id")
#user_revenue.filter(func.col("segment_id") == 1283).show(100,False)


In [9]:
percentile_revenue = user_full_.filter(func.col("revenue")>0).select("offer_type","frequency_label","test_variant_flag","revenue")\
                                .groupby("offer_type","frequency_label","test_variant_flag")\
                                 .agg(func.expr('percentile_approx(revenue,0.97)').alias('revenue_95_percentile'),
                                     func.max('revenue').alias('max_revenue'))
percc = spark.createDataFrame(percentile_revenue.rdd, percentile_revenue.schema)

users_percentile = user_full_.alias("a").join(percc.alias("b"),
                                              [func.col("a.frequency_label") == func.col("b.frequency_label"),
                                               func.col("a.offer_type") == func.col("b.offer_type"),
                                               func.col("a.test_variant_flag") == func.col("b.test_variant_flag")])\
                        .drop(func.col("b.frequency_label")).drop(func.col("b.offer_type")).drop(func.col("b.test_variant_flag"))\
                        .withColumn("truncated_revenue",func.when(func.col("revenue") > func.col("revenue_95_percentile"),
                                            func.col("revenue_95_percentile")).otherwise(func.col("revenue")))

user_revenue = users_percentile.groupby("segment_id","offer_id","test_variant_flag")\
            .agg(func.avg("truncated_revenue").alias("avg_revenue"),
                 func.max("max_revenue").alias("max_revenue"),
                 func.max("revenue_95_percentile").alias("revenue_95_percentile"),
                 func.stddev_samp("truncated_revenue").alias("std_revenue"),
                 func.count("truncated_revenue").alias("cust_count"))


uss = spark.createDataFrame(user_revenue.rdd, user_revenue.schema)
uss.registerTempTable("uss")

In [10]:
control_users = user_full_.filter(func.col("test_variant_flag") == 0)
non_control_users = user_full_.filter(func.col("test_variant_flag") == 1)

offer_list = non_control_users.select("segment_id","offer_id","start_date").distinct()\
                        .groupby("segment_id","offer_id")\
                        .agg(func.concat_ws(",",func.sort_array(func.collect_list("start_date"))).alias("collected_start_date"))

user_full_non_control = non_control_users.alias("a").join(offer_list.alias("b"), [func.col("a.offer_id") == func.col("b.offer_id"),
                                                                                  func.col("a.segment_id") == func.col("b.segment_id")])\
                                                .drop(func.col("b.offer_id")).drop(func.col("b.segment_id"))

date_list = user_full_non_control.select("start_date","segment_id","collected_start_date").distinct()

user_full_control = control_users.alias("a").join(date_list.alias("b"), [func.col("a.start_date") == func.col("b.start_date"),
                                                                        func.col("a.segment_id") == func.col("b.segment_id")])\
                                                .drop(func.col("b.start_date")).drop(func.col("b.segment_id"))
user_full = user_full_non_control.union(user_full_control)

percentile_revenue = user_full.filter(func.col("revenue")>0).select("offer_type","frequency_label","test_variant_flag","revenue")\
                                .groupby("offer_type","frequency_label","test_variant_flag")\
                                 .agg(func.expr('percentile_approx(revenue,0.97)').alias('revenue_95_percentile'),
                                     func.max('revenue').alias('max_revenue'))
percc = spark.createDataFrame(percentile_revenue.rdd, percentile_revenue.schema)

users_percentile = user_full.alias("a").join(percc.alias("b"),
                                              [func.col("a.frequency_label") == func.col("b.frequency_label"),
                                               func.col("a.offer_type") == func.col("b.offer_type"),
                                               func.col("a.test_variant_flag") == func.col("b.test_variant_flag")])\
                        .drop(func.col("b.frequency_label")).drop(func.col("b.offer_type")).drop(func.col("b.test_variant_flag"))\
                        .withColumn("truncated_revenue",func.when(func.col("revenue") > func.col("revenue_95_percentile"),
                                            func.col("revenue_95_percentile")).otherwise(func.col("revenue")))

user_revenue = users_percentile.groupby("collected_start_date","segment_id","offer_id","test_variant_flag")\
            .agg(func.avg("truncated_revenue").alias("avg_revenue"),
                 func.stddev_samp("truncated_revenue").alias("std_revenue"),
                 func.count("truncated_revenue").alias("cust_count"))

uss = spark.createDataFrame(user_revenue.rdd, user_revenue.schema)
uss.registerTempTable("uss")



In [ ]:
user_nir = spark.sql('''
select
	a.segment_id
	,a.offer_id
	,a.avg_revenue as test_revenue
	,b.avg_revenue as control_revenue
	,a.std_revenue as test_std
	,b.std_revenue as control_std
	,a.cust_count as test_cust_count
	,b.cust_count as control_cust_count
	,(a.avg_revenue - b.avg_revenue) as nir
	,sqrt(pow(a.std_revenue,2)/a.cust_count + pow(b.std_revenue,2)/b.cust_count) as diff_std
	,(a.avg_revenue - b.avg_revenue) - 1.282 * sqrt(pow(a.std_revenue,2)/a.cust_count + pow(b.std_revenue,2)/b.cust_count) as lower_ci_limit
	,(a.avg_revenue - b.avg_revenue) + 1.282 * sqrt(pow(a.std_revenue,2)/a.cust_count + pow(b.std_revenue,2)/b.cust_count) as higher_ci_limit
from uss a
join uss b
on a.segment_id = b.segment_id
where a.test_variant_flag = 1
	and b.test_variant_flag = 0
''')

model_nir = user_nir.select("segment_id"
                            , "offer_id"
                            , "test_revenue"
                            , "control_revenue"
                            , "test_std"
                            , "control_std"
                            , "diff_std"
                            , "test_cust_count"
                            , "control_cust_count"
                            , "nir"
                            , "lower_ci_limit"
                            , "higher_ci_limit")

user_full = user_seg.join(variant, [user_seg.variant_name == variant.name])\
    .join(measure, [user_seg.sid == measure.sid, user_seg.variant_name == measure.cell_name])\
    .select(user_seg.sid,
            user_seg.offer_id,
            user_seg.segment_id,
            measure.tier,
            func.when(measure.tier == 2, measure.stars_awarded).otherwise(func.lit(0)).alias("stars_awarded")
           )

model_star =  user_full.groupby("offer_id","segment_id").agg(func.avg("stars_awarded").alias("avg_stars_awarded"))


model_input = model_nir.join(model_star, [model_nir.segment_id == model_star.segment_id, model_nir.offer_id == model_star.offer_id])\
                        .join(model_user,[model_nir.segment_id == model_user.segment_id])\
                        .join(offer_lookup,[model_nir.offer_id == offer_lookup.offer_id])\
                        .select(model_nir.segment_id.alias("microsegment")
                                ,model_nir.offer_id.alias("offer")
                                ,model_nir.test_revenue
                                ,model_nir.control_revenue
                                ,model_nir.test_std
                                ,model_nir.control_std
                                ,model_nir.diff_std
                                ,model_nir.test_cust_count.alias("previous_customers")
                                ,model_nir.control_cust_count.alias("local_control_customers")
                                ,model_nir.nir.alias("expected_nir")
                                ,model_nir.lower_ci_limit
                                ,model_nir.higher_ci_limit
                                ,model_star.avg_stars_awarded
                                ,model_user.user_count.alias("customers")
                                ,offer_lookup.offer_type)

df_input = model_input.toPandas()
df_input.dtypes

# pull in the global variables
offer_type = df_input['offer_type'].unique().tolist()
total_budget = range(2000000, 4000001, 500000)
star_value = decimal.Decimal('0.037')

# initialize a results data frame
df_full_results = pd.DataFrame()

# get current time and run id
current = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
run_id = str(uuid.uuid4())

# loop through the buckets and optimize
# ------------------------------------------------------

for i_offer_type in offer_type:
    for i_total_budget in total_budget:
        # filter out eligible offers
        df_offer_full = df_input[(df_input['offer_type'] == i_offer_type)]

        # and calculate the population-weighted cost, response & nir
        df_offer_full['segment_cost'] = df_offer_full['customers'] * df_offer_full['avg_stars_awarded'] * star_value
        df_offer_full['segment_nir'] = df_offer_full['customers'] * df_offer_full['expected_nir']

        # add budget
        df_offer_full['total_budget'] = i_total_budget
        # add current time
        df_offer_full['update_time'] = current
        df_offer_full['run_id'] = run_id

        # Apply business rules: filter out negative NIR and less than 200 previous customers
        df_offer = df_offer_full[
            (df_offer_full['expected_nir'] >= 0) & (df_offer_full['previous_customers'] >= 2000) & (
            df_offer_full['local_control_customers'] >= 2000)]

        # create list of all possible segment/offer combinations
        combos = df_offer[['microsegment', 'offer']]
        # combos.itertuples(index=False, name=None)
        possible_offers = [tuple(x) for x in combos.itertuples(index=False, name=None)]

        # dictionary of expected offer segment nir
        combos = df_offer[['microsegment', 'offer', 'segment_nir']]
        # combos.itertuples(index=False, name=None)
        offer_nir_values = {x[0:2]: x[2] for x in combos.itertuples(index=False, name=None)}

        # dictionary of expected offer costs
        combos = df_offer[['microsegment', 'offer', 'segment_cost']]
        # combos.itertuples(index=False, name=None)
        offer_costs = {x[0:2]: x[2] for x in combos.itertuples(index=False, name=None)}

        o = pulp.LpVariable.dicts('', possible_offers,
                                  lowBound=0,
                                  upBound=1,
                                  cat=pulp.LpInteger)

        # formulate and solve the optimization problem via pulp
        # ------------------------------------------------------

        # create the problem
        prob = pulp.LpProblem('maximize nir', pulp.LpMaximize)

        # add the objective function first
        prob += bt.sum([float(offer_nir_values[offer]) * o[offer] for offer in
                        possible_offers])  # <--- eval statement switches values used in objective function

        # add in the total budget constraint
        prob += bt.sum([float(offer_costs[offer]) * o[offer] for offer in possible_offers]) <= i_total_budget, ''

        # each segment gets exactly one offer
        segments = df_offer['microsegment'].tolist()
        for segment in segments:
            offers = df_offer[(df_offer['microsegment'] == segment)]['offer'].tolist()
            prob += pulp.lpSum([o[(segment, offer)] for offer in offers]) == 1, ''

        # write the problem to an lp file
        # prob.writeLP(path+'offer_optimization_nir('+str(i_offer_type)+'_'+str(i_total_budget)+').lp')

        # solve the lp
        prob.solve()

        # print the results status
        print 'status nir optimization (' + str(i_offer_type) + ', ' + str(i_total_budget) + '): ' + pulp.LpStatus[
            prob.status].lower()

        # save the decision variables with their optimum value
        results = {}
        for v in prob.variables():
            if str(v.name) != "__dummy":
                results[eval(str(v.name).replace("_", ""))] = v.varValue

        # convert decision variable dictionary to a dataframe
        df_opt_results = pd.DataFrame.from_dict(results, orient='index')
    
        df_opt_results.columns = ['chosen_nir']

        # process the output
        # ------------------------------------------------------

        # join the results to the original full data frame
        df_opt_results.reset_index(drop=False, inplace=True)
        df_opt_results.columns = ['index_full', 'chosen']
        df_opt_results[['microsegment', 'offer']] = df_opt_results.index_full.apply(pd.Series)
        df_opt_results.drop('index_full', axis=1, inplace=True)

        df_full_results_interim = pd.merge(df_offer_full, df_opt_results, how='left', on=['microsegment', 'offer'])
        df_full_results_interim['optimization_status'] = pulp.LpStatus[prob.status].lower()

        # and combine with the other optimization runs
        df_full_results = pd.concat([df_full_results, df_full_results_interim])

        df_full_results = df_full_results[[
                    'microsegment'
                    , 'offer'
                    , 'expected_nir'
                    , 'avg_stars_awarded'
                    , 'customers'
                    , 'previous_customers'
                    , 'local_control_customers'
                    , 'offer_type'
                    , 'chosen'
                    , 'segment_cost'
                    , 'segment_nir'
                    , 'total_budget'
                    , 'optimization_status'
                    , 'update_time'
                    , 'run_id'
                    , 'test_revenue'
                    , 'control_revenue'
                    , 'test_std'
                    , 'control_std'
                    , 'lower_ci_limit'
                    , 'higher_ci_limit'
                    , 'diff_std']]

# save results
# ------------------------------------------------------

df_full_results_to_save_temp = spark.createDataFrame(df_full_results)
df_full_results_to_save = df_full_results_to_save_temp.select(
		"microsegment"
		,"offer"
		,"expected_nir"
		,"avg_stars_awarded"
		,"customers"
		,"previous_customers"
		,"local_control_customers"
		,"offer_type"
		,func.regexp_replace("chosen", 'NaN', '').alias("chosen")
		,"segment_cost"
		,"segment_nir"
		,"total_budget"
		,"optimization_status"
		,"update_time"
		,"run_id"
		,"test_revenue"
		,"control_revenue"
		,func.regexp_replace("test_std", 'NaN', '').alias("test_std")
		,func.regexp_replace("control_std", 'NaN', '').alias("control_std")
		,func.regexp_replace("lower_ci_limit", 'NaN', '').alias("lower_ci_limit")
		,func.regexp_replace("higher_ci_limit", 'NaN', '').alias("higher_ci_limit")
		,func.regexp_replace("diff_std", 'NaN', '').alias("diff_std"))

outfile_df_full_results_to_save = "s3://stx-apollo-pr-datascience-internal/eosoba/optimization_v1.2/%s/final_result_outlier_97/" % latest_date
subprocess.call(["aws", "s3", "rm", outfile_df_full_results_to_save, "--recursive"])
df_full_results_to_save.repartition(1).write.csv(path=outfile_df_full_results_to_save, sep='|', header='true',
                                                 nullValue='', quote=None,mode="overwrite")


In [ ]:
fn = spark.read.csv("s3://stx-apollo-pr-datascience-internal/prod/optimization_v1.2/%s/final_result" % latest_date,header=True,sep="|")
path = "s3://stx-apollo-pr-datascience-internal/eosoba/optimization_v1.2/%s/final_result/" % latest_date
fn.repartition(1).write.csv(path,header=True,sep="|",nullValue='', quote=None,mode="overwrite")
    

In [11]:
user_nir = spark.sql('''
select
	a.segment_id
	,a.collected_start_date
	,a.offer_id
	,a.avg_revenue as test_revenue
	,b.avg_revenue as control_revenue
	,a.std_revenue as test_std
	,b.std_revenue as control_std
	,a.cust_count as test_cust_count
	,b.cust_count as control_cust_count
	,(a.avg_revenue - b.avg_revenue) as nir
	,sqrt(pow(a.std_revenue,2)/a.cust_count + pow(b.std_revenue,2)/b.cust_count) as diff_std
	,(a.avg_revenue - b.avg_revenue) - 1.282 * sqrt(pow(a.std_revenue,2)/a.cust_count + pow(b.std_revenue,2)/b.cust_count) as lower_ci_limit
	,(a.avg_revenue - b.avg_revenue) + 1.282 * sqrt(pow(a.std_revenue,2)/a.cust_count + pow(b.std_revenue,2)/b.cust_count) as higher_ci_limit
from uss a
join uss b
on a.segment_id = b.segment_id and  a.collected_start_date = b.collected_start_date
where a.test_variant_flag = 1 and b.test_variant_flag = 0
''')


user_full = user_seg.join(variant, [user_seg.variant_name == variant.name])\
    .join(measure, [user_seg.sid == measure.sid, user_seg.variant_name == measure.cell_name])\
    .select(user_seg.sid,
            user_seg.offer_id,
            user_seg.segment_id,
            measure.tier,
            func.when(measure.tier == 2, measure.stars_awarded).otherwise(func.lit(0)).alias("stars_awarded")
           )

model_star =  user_full.groupby("offer_id","segment_id").agg(func.avg("stars_awarded").alias("avg_stars_awarded"))


model_nir = user_nir.select("segment_id"
                            ,"collected_start_date"
                            , "offer_id"
                            , "test_revenue"
                            , "control_revenue"
                            , "test_std"
                            , "control_std"
                            , "diff_std"
                            , "test_cust_count"
                            , "control_cust_count"
                            , "nir"
                            , "lower_ci_limit"
                            , "higher_ci_limit")


model_input = model_nir.join(model_star, [model_nir.segment_id == model_star.segment_id, model_nir.offer_id == model_star.offer_id])\
                        .join(model_user,[model_nir.segment_id == model_user.segment_id])\
                        .join(offer_lookup,[model_nir.offer_id == offer_lookup.offer_id])\
                        .select(model_nir.segment_id.alias("microsegment")
                                ,model_nir.offer_id.alias("offer")
                                ,model_nir.collected_start_date
                                ,model_nir.test_revenue
                                ,model_nir.control_revenue
                                ,model_nir.test_std
                                ,model_nir.control_std
                                ,model_nir.diff_std
                                ,model_nir.test_cust_count.alias("previous_customers")
                                ,model_nir.control_cust_count.alias("local_control_customers")
                                ,model_nir.nir.alias("expected_nir")
                                ,model_nir.lower_ci_limit
                                ,model_nir.higher_ci_limit
                                ,model_star.avg_stars_awarded
                                ,model_user.user_count.alias("customers")
                                ,offer_lookup.offer_type)



df_input = model_input.toPandas()
df_input.dtypes

# pull in the global variables
offer_type = df_input['offer_type'].unique().tolist()
total_budget = range(2000000, 4000001, 500000)
star_value = decimal.Decimal('0.037')

# initialize a results data frame
df_full_results = pd.DataFrame()

# get current time and run id
current = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
run_id = str(uuid.uuid4())

# loop through the buckets and optimize
# ------------------------------------------------------

for i_offer_type in offer_type:
    for i_total_budget in total_budget:
        # filter out eligible offers
        df_offer_full = df_input[(df_input['offer_type'] == i_offer_type)]

        # and calculate the population-weighted cost, response & nir
        df_offer_full['segment_cost'] = df_offer_full['customers'] * df_offer_full['avg_stars_awarded'] * star_value
        df_offer_full['segment_nir'] = df_offer_full['customers'] * df_offer_full['expected_nir']

        # add budget
        df_offer_full['total_budget'] = i_total_budget
        # add current time
        df_offer_full['update_time'] = current
        df_offer_full['run_id'] = run_id

        # Apply business rules: filter out negative NIR and less than 200 previous customers
        df_offer = df_offer_full[
            (df_offer_full['expected_nir'] >= 0) & (df_offer_full['previous_customers'] >= 2000) & (
            df_offer_full['local_control_customers'] >= 2000)]

        # create list of all possible segment/offer combinations
        combos = df_offer[['microsegment', 'offer']]
        # combos.itertuples(index=False, name=None)
        possible_offers = [tuple(x) for x in combos.itertuples(index=False, name=None)]

        # dictionary of expected offer segment nir
        combos = df_offer[['microsegment', 'offer', 'segment_nir']]
        # combos.itertuples(index=False, name=None)
        offer_nir_values = {x[0:2]: x[2] for x in combos.itertuples(index=False, name=None)}

        # dictionary of expected offer costs
        combos = df_offer[['microsegment', 'offer', 'segment_cost']]
        # combos.itertuples(index=False, name=None)
        offer_costs = {x[0:2]: x[2] for x in combos.itertuples(index=False, name=None)}

        o = pulp.LpVariable.dicts('', possible_offers,
                                  lowBound=0,
                                  upBound=1,
                                  cat=pulp.LpInteger)

        # formulate and solve the optimization problem via pulp
        # ------------------------------------------------------

        # create the problem
        prob = pulp.LpProblem('maximize nir', pulp.LpMaximize)

        # add the objective function first
        prob += bt.sum([float(offer_nir_values[offer]) * o[offer] for offer in
                        possible_offers])  # <--- eval statement switches values used in objective function

        # add in the total budget constraint
        prob += bt.sum([float(offer_costs[offer]) * o[offer] for offer in possible_offers]) <= i_total_budget, ''

        # each segment gets exactly one offer
        segments = df_offer['microsegment'].tolist()
        for segment in segments:
            offers = df_offer[(df_offer['microsegment'] == segment)]['offer'].tolist()
            prob += pulp.lpSum([o[(segment, offer)] for offer in offers]) == 1, ''

        # write the problem to an lp file
        # prob.writeLP(path+'offer_optimization_nir('+str(i_offer_type)+'_'+str(i_total_budget)+').lp')

        # solve the lp
        prob.solve()

        # print the results status
        print 'status nir optimization (' + str(i_offer_type) + ', ' + str(i_total_budget) + '): ' + pulp.LpStatus[
            prob.status].lower()

        # save the decision variables with their optimum value
        results = {}
        for v in prob.variables():
            if str(v.name) != "__dummy":
                results[eval(str(v.name).replace("_", ""))] = v.varValue

        # convert decision variable dictionary to a dataframe
        df_opt_results = pd.DataFrame.from_dict(results, orient='index')
    
        df_opt_results.columns = ['chosen_nir']

        # process the output
        # ------------------------------------------------------

        # join the results to the original full data frame
        df_opt_results.reset_index(drop=False, inplace=True)
        df_opt_results.columns = ['index_full', 'chosen']
        df_opt_results[['microsegment', 'offer']] = df_opt_results.index_full.apply(pd.Series)
        df_opt_results.drop('index_full', axis=1, inplace=True)

        df_full_results_interim = pd.merge(df_offer_full, df_opt_results, how='left', on=['microsegment', 'offer'])
        df_full_results_interim['optimization_status'] = pulp.LpStatus[prob.status].lower()

        # and combine with the other optimization runs
        df_full_results = pd.concat([df_full_results, df_full_results_interim])

        df_full_results = df_full_results[[
                    'microsegment'
                    , 'offer'
                    , 'expected_nir'
                    , 'avg_stars_awarded'
                    , 'customers'
                    , 'previous_customers'
                    , 'local_control_customers'
                    , 'offer_type'
                    , 'chosen'
                    , 'collected_start_date'
                    , 'segment_cost'
                    , 'segment_nir'
                    , 'total_budget'
                    , 'optimization_status'
                    , 'update_time'
                    , 'run_id'
                    , 'test_revenue'
                    , 'control_revenue'
                    , 'test_std'
                    , 'control_std'
                    , 'lower_ci_limit'
                    , 'higher_ci_limit'
                    , 'diff_std']]

# save results
# ------------------------------------------------------

df_full_results_to_save_temp = spark.createDataFrame(df_full_results)
df_full_results_to_save = df_full_results_to_save_temp.select(
		"microsegment"
		,"offer"
		,"expected_nir"
		,"avg_stars_awarded"
		,"customers"
		,"previous_customers"
		,"local_control_customers"
		,"offer_type"
		,func.regexp_replace("chosen", 'NaN', '').alias("chosen")
        ,"collected_start_date"
		,"segment_cost"
		,"segment_nir"
		,"total_budget"
		,"optimization_status"
		,"update_time"
		,"run_id"
		,"test_revenue"
		,"control_revenue"
		,func.regexp_replace("test_std", 'NaN', '').alias("test_std")
		,func.regexp_replace("control_std", 'NaN', '').alias("control_std")
		,func.regexp_replace("lower_ci_limit", 'NaN', '').alias("lower_ci_limit")
		,func.regexp_replace("higher_ci_limit", 'NaN', '').alias("higher_ci_limit")
		,func.regexp_replace("diff_std", 'NaN', '').alias("diff_std"))

outfile_df_full_results_to_save = "s3://stx-apollo-pr-datascience-internal/eosoba/optimization_v1.2/%s/final_result_control_logic_outlier_97/" % latest_date
subprocess.call(["aws", "s3", "rm", outfile_df_full_results_to_save, "--recursive"])
df_full_results_to_save.repartition(1).write.csv(path=outfile_df_full_results_to_save, sep='|', header='true',
                                                 nullValue='', quote=None,mode="overwrite")

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

status nir optimization (Dash 7-day, 2000000): optimal
status nir optimization (Dash 7-day, 2500000): optimal


/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:178: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



status nir optimization (Dash 7-day, 3000000): optimal
status nir optimization (Dash 7-day, 3500000): optimal
status nir optimization (Dash 7-day, 4000000): optimal
status nir optimization (Streak 7-day 1-hurdle, 2000000): optimal
status nir optimization (Streak 7-day 1-hurdle, 2500000): optimal
status nir optimization (Streak 7-day 1-hurdle, 3000000): optimal
status nir optimization (Streak 7-day 1-hurdle, 3500000): optimal
status nir optimization (Streak 7-day 1-hurdle, 4000000): optimal
status nir optimization (Streak 7-day 3-hurdle, 2000000): optimal
status nir optimization (Streak 7-day 3-hurdle, 2500000): optimal
status nir optimization (Streak 7-day 3-hurdle, 3000000): optimal
status nir optimization (Streak 7-day 3-hurdle, 3500000): optimal
status nir optimization (Streak 7-day 3-hurdle, 4000000): optimal
status nir optimization (Dash 7-day 2-hurdle, 2000000): optimal
status nir optimization (Dash 7-day 2-hurdle, 2500000): optimal
status nir optimization (Dash 7-day 2-hurdle, 3

In [ ]:
path ="s3://stx-apollo-pr-datascience-internal/eosoba/optimization_v1.2/{0}/final_result_outlier_positive".format(latest_date)
ff = spark.read.csv(path,header=True,sep="|")
pp = path+"_1"
ff.repartition(1).write.csv(pp,header=True,sep="|",mode="overwrite")

In [ ]:
s3://stx-apollo-pr-datascience-internal/eosoba/optimization_v1.2/20181105/final_result_control_logic_outlier/